<a href="https://colab.research.google.com/github/cyrus2281/notes/blob/main/MachineLearning/Recommenders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommenders

## Mathematical Recommenders

### Lift

$$
\text{Lift} = \frac{p(A,B)}{p(A)p(B)} = \frac{p(A|B)}{p(A)} = \frac{p(B|A)}{p(B)}
$$

- Symmetric
- If A and B are independent, then $p(A|B) = p(A)$
  - $p(A|B) / p(A) = 1$
- if increasing the probability of B increases the probability of A, then Lift > 1

### Hacker News Formula

Balancing Popularirty with Age

$$
\frac{f(\text{popularity})}{g(\text{age})} \\[1cm]
$$

$$
\text{score} = \frac{(\text{ups} - \text{downs} -1 )^{0.8}}{(\text{age}+2)^{\text{gravity}}} \times \text{penalty}
$$

- gravity = 1.8
- penalty = multiplier to implement "business rules" (e.g. penalize self-posts, "controversial" posts, + many more rules)

\

age starts from 2, to prevent division by zero

exponent of numerator is bigger than the exponent of the denominator, meaning denominator grows faster.

Age always overtakes popularity

\

exponent 0.8 causes sublinear growth.
meaning 0 → 100 worth more than 1000 → 1100



### Reddit Formula

$$
\text{score} = \text{sign}(\text{ups}-\text{down}) \times \log \{ \max(1, |\text{ups} - \text{downs}|) \} + \frac{\text{age}}{45000}
$$

- log of the absoulte value of net votes - sublinear curve - initial votes matter more - max since log 0 is not possible

\

- can be positive or negative
  - The more downvotes you get, the futher your score goes down

\

- age is in seconds from inception of reddit
- age is always positive
- newer links → more score
- reddit scores will forever increase linearly


## Statistics

### Smoothing (Dampening)

To resolve the issue of 0 in sample data when getting mean

$$
r = \frac{\sum^N_{i=1} X_i + \lambda \mu_0}{N+\lambda}
$$


- $\lambda$ some random small non-zero number
- $\mu_0$ the global avergage or just some middle value

\

For example:
- 1000 reviews of 4 star - μ = 3 - λ = 1 → 3.999
- 5 reviews of 4 star - μ = 3 - λ = 1 → 3.83
- 1 review of 4 star - μ = 3 - λ = 1 → 3.5

### Explore-Exploit Dilemma

Example 1

Imagine we want to find the slot machine with the highest win rate among 10 slot machines.

Traditional statistical test can tell us whether or not there's a significant difference between win rates between machines.

If playing each machine 100 times, meaning 1000 turns total, 900 (9/10) turns yielded a suboptimal reward.

Hence the dilemma, Play more or play less!


---

Example 2

Watching a bunach of YouTube videos on how to make eggs.

Now your reccomendations are filled with videos about making eggs

porbably suboptimal - once I've figured out how to make eggs, I don't want to watch more egg videos.

YouTube is not exploiting the fact that I watched eggs video and not exploring other topics

Should there be a stronger exploration component?

Maybe I'd like to seE movie trailers or machine learning videos

---

How do we strike a balance between these 2 opposing forces?

Smoothed average gives us one part of the solution

Making good things look worse and bad things look better


### Bayesian Method

Bayesian method automatically balances need to explore and exploit

- 2 fat distributions: explore both (totally random ranking)
- 2 skinny distributions: exploit both (nearly deterministic ranking)
- Mixed: explore and exploit co-exit


Completely automatic - does not require A/B testing

